# 🎓 ATLAS: Academic Task and Learning Agent System

## 📖 한글 학습용 완전판

---

### 🎯 이 노트북에서 배울 내용

**ATLAS**는 학생들의 학업 관리를 돕는 **지능형 멀티 에이전트 시스템**입니다.

#### 핵심 기능:
- 📅 **자동 일정 관리**: AI가 시험, 과제 마감일을 분석하여 최적의 학습 계획 수립
- 📝 **강의 자료 요약**: 방대한 강의 내용을 핵심만 추출하여 학습 노트 생성
- 💡 **개인화된 학습 조언**: 학생의 학습 스타일에 맞춘 맞춤형 조언 제공
- 🤖 **다중 에이전트 협업**: 여러 전문 AI가 협력하여 최적의 결과 도출

---

### 🏗️ 시스템 아키텍처

```
                    ┌─────────────────────────┐
                    │   Coordinator Agent     │  ← 마스터 조율자
                    │    (요청 분석 및 위임)   │
                    └───────────┬─────────────┘
                                │
                ┌───────────────┼───────────────┐
                │               │               │
        ┌───────▼──────┐ ┌─────▼──────┐ ┌─────▼──────┐
        │   Planner    │ │ NoteWriter │ │  Advisor   │
        │   (계획)     │ │  (작성)    │ │  (조언)    │
        └──────────────┘ └────────────┘ └────────────┘
```

---

### 📚 학습 목표

이 노트북을 통해 다음을 배우게 됩니다:

1. **ReACT 패턴** (Reasoning + Acting)
   - AI가 사고하고 행동하는 구조화된 방법
   - Thought → Action → Observation 순환

2. **멀티 에이전트 시스템**
   - 여러 전문 에이전트의 협업 구조
   - Coordinator를 통한 작업 분배

3. **LangGraph 워크플로우**
   - 상태 기반 그래프 구성
   - 에이전트 간 데이터 흐름 관리

4. **실전 응용**
   - 실제 학업 시나리오 해결
   - 시간 관리, 자료 처리, 계획 수립

---

### ⚠️ 사전 준비

- Python 3.8+
- NVIDIA API 키 (NeMo 모델 사용)
- 필수 라이브러리: `langgraph`, `langchain`, `openai`, `pydantic`

---

### 🚀 시작하기

아래 셀들을 순서대로 실행하면서 학습을 진행하세요!

---

In [ ]:
class LLMConfig:
    """
    LLM 설정을 중앙에서 관리하는 클래스
    
    모든 에이전트가 동일한 설정을 공유합니다.
    """
    # NVIDIA API 엔드포인트
    BASE_URL = "https://integrate.api.nvidia.com/v1"
    
    # 사용할 모델 (Nemotron-4 340B)
    MODEL_NAME = "nvidia/nemotron-4-340b-instruct"
    
    # 최대 토큰 수
    MAX_TOKENS = 2048
    
    # Temperature (창의성 조절: 0.0 = 결정적, 1.0 = 창의적)
    TEMPERATURE = 0.7
    
    @classmethod
    def get_client(cls, api_key: str) -> AsyncOpenAI:
        """
        비동기 OpenAI 클라이언트 생성
        
        Args:
            api_key: NVIDIA API 키
            
        Returns:
            AsyncOpenAI: 비동기 클라이언트 인스턴스
        """
        return AsyncOpenAI(
            base_url=cls.BASE_URL,
            api_key=api_key
        )

# 전역 LLM 클라이언트 생성
llm_client = LLMConfig.get_client(NEMOTRON_4_340B_INSTRUCT_KEY)

print("✅ LLM 클라이언트 생성 완료")
print(f"📊 모델: {LLMConfig.MODEL_NAME}")
print(f"🌡️  Temperature: {LLMConfig.TEMPERATURE}")

In [ ]:
class DataManager:
    """
    데이터 관리 클래스
    
    학생 프로필, 캘린더, 할 일 목록을 중앙에서 관리합니다.
    모든 에이전트는 이 클래스를 통해 데이터에 접근합니다.
    """
    
    def __init__(self):
        """데이터 저장소 초기화 (모두 None으로 시작)"""
        self.profile_data = None
        self.calendar_data = None
        self.task_data = None
    
    def load_data(self, profile_json: str, calendar_json: str, task_json: str):
        """
        JSON 문자열을 파싱하여 데이터 로드
        
        Args:
            profile_json: 학생 프로필 JSON
            calendar_json: 캘린더 이벤트 JSON
            task_json: 할 일 목록 JSON
        """
        self.profile_data = json.loads(profile_json)
        self.calendar_data = json.loads(calendar_json)
        self.task_data = json.loads(task_json)
    
    def get_student_profile(self, student_id: str) -> Dict:
        """
        학생 ID로 프로필 검색
        
        Args:
            student_id: 학생 고유 ID
        
        Returns:
            Dict: 학생 프로필 (없으면 None)
        
        구현 방식:
            - generator expression + next()로 효율적 검색
            - 전체 리스트 순회를 피함
        """
        if self.profile_data:
            return next((p for p in self.profile_data["profiles"]
                        if p["id"] == student_id), None)
        return None
    
    def parse_datetime(self, dt_str: str) -> datetime:
        """
        스마트 날짜/시간 파싱 (타임존 처리 포함)
        
        Args:
            dt_str: ISO 형식 날짜/시간 문자열
        
        Returns:
            datetime: UTC 타임존으로 변환된 datetime 객체
        
        처리 방식:
            1. 타임존 정보가 있으면 파싱 후 UTC로 변환
            2. 타임존 정보가 없으면 UTC로 가정
        """
        try:
            # 'Z'를 '+00:00'으로 변환하여 파싱
            dt = datetime.fromisoformat(dt_str.replace('Z', '+00:00'))
            return dt.astimezone(timezone.utc)
        except ValueError:
            # 타임존 없으면 UTC로 가정
            dt = datetime.fromisoformat(dt_str)
            return dt.replace(tzinfo=timezone.utc)
    
    def get_upcoming_events(self, days: int = 7) -> List[Dict]:
        """
        향후 N일간의 일정 조회
        
        Args:
            days: 조회할 기간 (기본값: 7일)
        
        Returns:
            List[Dict]: 시간순으로 정렬된 이벤트 목록
        
        구현 로직:
            - 현재 시각(UTC)부터 days일 후까지의 이벤트만 필터링
            - 잘못된 데이터는 warning 출력 후 스킵
        """
        if not self.calendar_data:
            return []
        
        now = datetime.now(timezone.utc)
        future = now + timedelta(days=days)
        
        events = []
        for event in self.calendar_data.get("events", []):
            try:
                start_time = self.parse_datetime(event["start"]["dateTime"])
                
                # 현재 ~ future 범위의 이벤트만 포함
                if now <= start_time <= future:
                    events.append(event)
            except (KeyError, ValueError) as e:
                print(f"⚠️  이벤트 처리 오류: {str(e)}")
                continue
        
        return events
    
    def get_active_tasks(self) -> List[Dict]:
        """
        미완료 과제 목록 조회
        
        Returns:
            List[Dict]: 활성 과제 목록 (파싱된 마감일 포함)
        
        필터링 조건:
            1. status가 'needsAction' (미완료)
            2. 마감일이 현재 시각보다 미래
        
        추가 기능:
            - 각 task에 'due_datetime' 필드 추가 (파싱된 datetime 객체)
        """
        if not self.task_data:
            return []
        
        now = datetime.now(timezone.utc)
        active_tasks = []
        
        for task in self.task_data.get("tasks", []):
            try:
                due_date = self.parse_datetime(task["due"])
                
                # 미완료 + 미래 마감일인 과제만 포함
                if task["status"] == "needsAction" and due_date > now:
                    # 파싱된 datetime 추가 (에이전트가 쉽게 사용 가능)
                    task["due_datetime"] = due_date
                    active_tasks.append(task)
            except (KeyError, ValueError) as e:
                print(f"⚠️  과제 처리 오류: {str(e)}")
                continue
        
        return active_tasks

print("✅ DataManager 클래스 정의 완료")

In [ ]:
class ReActAgent:
    """
    ReACT 패턴 기반 에이전트 베이스 클래스
    
    모든 전문 에이전트(Planner, NoteWriter, Advisor)의 부모 클래스입니다.
    공통 도구와 기본 로직을 제공합니다.
    """
    
    def __init__(self, llm):
        """
        Args:
            llm: 언어 모델 인스턴스
        """
        self.llm = llm
        
        # Few-shot 학습 예제 저장소 (나중에 사용)
        self.few_shot_examples = []
        
        # 사용 가능한 도구 딕셔너리
        self.tools = {
            "search_calendar": self.search_calendar,
            "analyze_tasks": self.analyze_tasks,
            "check_learning_style": self.check_learning_style,
            "check_performance": self.check_performance
        }
    
    async def search_calendar(self, state: AcademicState) -> List[Dict]:
        """
        캘린더에서 향후 이벤트 검색
        
        Returns:
            미래 이벤트 목록 (현재 시각 이후)
        """
        events = state["calendar"].get("events", [])
        now = datetime.now(timezone.utc)
        
        # 미래 이벤트만 필터링
        return [e for e in events 
                if datetime.fromisoformat(e["start"]["dateTime"]) > now]
    
    async def analyze_tasks(self, state: AcademicState) -> List[Dict]:
        """
        할 일 목록 분석
        
        Returns:
            전체 과제 목록
        """
        return state["tasks"].get("tasks", [])
    
    async def check_learning_style(self, state: AcademicState) -> AcademicState:
        """
        학생의 학습 스타일과 패턴 조회
        
        Returns:
            학습 스타일 정보가 추가된 state
        """
        profile = state["profile"]
        
        # 학습 선호도 추출
        learning_data = {
            "style": profile.get("learning_preferences", {}).get("learning_style", {}),
            "patterns": profile.get("learning_preferences", {}).get("study_patterns", {})
        }
        
        # 결과를 state에 저장
        if "results" not in state:
            state["results"] = {}
        state["results"]["learning_analysis"] = learning_data
        
        return state
    
    async def check_performance(self, state: AcademicState) -> AcademicState:
        """
        현재 학업 성취도 확인
        
        Returns:
            성적 정보가 추가된 state
        """
        profile = state["profile"]
        
        # 수강 중인 과목 정보 추출
        courses = profile.get("academic_info", {}).get("current_courses", [])
        
        # 결과를 state에 저장
        if "results" not in state:
            state["results"] = {}
        state["results"]["performance_analysis"] = {"courses": courses}
        
        return state

print("✅ ReActAgent 클래스 정의 완료")

In [ ]:
# ===================================
# 1. analyze_context: 학생 상황 분석
# ===================================

async def analyze_context(state: AcademicState) -> Dict:
    """
    학생의 현재 상황을 분석하여 Coordinator에게 제공
    
    Returns:
        학생 프로필, 일정, 과제, 학습 선호도 등을 포함한 딕셔너리
    """
    profile = state.get("profile", {})
    calendar = state.get("calendar", {})
    tasks = state.get("tasks", {})
    messages = state.get("messages", [])
    
    # 최근 메시지에서 현재 과목 추출
    current_course = None
    if messages:
        last_message = messages[-1].content.lower()
        # 간단한 키워드 매칭 (실제로는 더 정교한 분석 필요)
        for course in profile.get("academic_info", {}).get("current_courses", []):
            if course.get("name", "").lower() in last_message:
                current_course = course
                break
    
    # 분석 결과 구조화
    context = {
        "student_info": {
            "name": profile.get("name", "Unknown"),
            "major": profile.get("academic_info", {}).get("major", "Unknown")
        },
        "learning_preferences": profile.get("learning_preferences", {}),
        "upcoming_events": len(calendar.get("events", [])),
        "active_tasks": len(tasks.get("tasks", [])),
        "current_course": current_course
    }
    
    return context

# ===================================
# 2. COORDINATOR_PROMPT: 시스템 프롬프트
# ===================================

COORDINATOR_PROMPT = """당신은 학업 지원 에이전트들을 조율하는 Coordinator입니다.

사용 가능한 에이전트:
• PLANNER: 일정 관리 및 시간 배분
• NOTEWRITER: 학습 자료 작성 및 요약
• ADVISOR: 개인화된 학습 조언

병렬 실행 규칙:
1. 독립적인 에이전트는 동시 실행 가능
2. 의존성 있는 작업은 순차 실행
3. 결과를 통합하여 사용자에게 제공

ReACT 패턴:
Thought: [요청 분석 및 필요한 지원 유형 파악]
Action: [최적의 에이전트 조합 선택]
Observation: [선택한 에이전트의 역량 평가]
Decision: [최종 에이전트 배치 계획]

분석 요소:
1. 작업 복잡도와 범위
2. 시간 제약
3. 필요 자원
4. 학습 스타일 적합성
5. 지원 유형

컨텍스트:
요청: {request}
학생 정보: {context}

응답 형식:
Thought: [학업 요구사항 및 상황 분석]
Action: [에이전트 선택 및 그룹화 전략]
Observation: [예상 워크플로우 및 의존성]
Decision: [최종 에이전트 배치 계획 및 근거]
"""

# ===================================
# 3. coordinator_agent: 메인 조율 함수
# ===================================

async def coordinator_agent(state: AcademicState) -> Dict:
    """
    ReACT 패턴으로 에이전트를 조율하는 핵심 함수
    
    처리 흐름:
    1. 학생 상황 분석 (analyze_context)
    2. LLM에게 요청 분석 위임
    3. 응답 파싱 및 구조화
    4. 에이전트 실행 계획 반환
    
    Returns:
        required_agents: 필요한 에이전트 목록
        priority: 우선순위
        concurrent_groups: 병렬 실행 그룹
    """
    try:
        # 1. 현재 상황 분석
        context = await analyze_context(state)
        query = state["messages"][-1].content
        
        # 2. LLM에게 조율 요청
        response = await llm_client.chat.completions.create(
            model=LLMConfig.MODEL_NAME,
            messages=[
                {"role": "system", "content": COORDINATOR_PROMPT.format(
                    request=query,
                    context=json.dumps(context, ensure_ascii=False, indent=2)
                )}
            ],
            max_tokens=LLMConfig.MAX_TOKENS,
            temperature=LLMConfig.TEMPERATURE
        )
        
        llm_response = response.choices[0].message.content
        
        # 3. 응답 파싱
        analysis = parse_coordinator_response(llm_response)
        
        return {
            "results": {
                "coordinator_analysis": {
                    "required_agents": analysis.get("required_agents", ["PLANNER"]),
                    "priority": analysis.get("priority", {"PLANNER": 1}),
                    "concurrent_groups": analysis.get("concurrent_groups", [["PLANNER"]]),
                    "reasoning": llm_response
                }
            }
        }
    
    except Exception as e:
        print(f"⚠️  Coordinator 오류: {e}")
        # 폴백: PLANNER만 실행
        return {
            "results": {
                "coordinator_analysis": {
                    "required_agents": ["PLANNER"],
                    "priority": {"PLANNER": 1},
                    "concurrent_groups": [["PLANNER"]],
                    "reasoning": "에러 발생으로 PLANNER로 폴백"
                }
            }
        }

# ===================================
# 4. parse_coordinator_response: 응답 파싱
# ===================================

def parse_coordinator_response(response: str) -> Dict:
    """
    LLM 응답을 파싱하여 실행 계획 추출
    
    파싱 로직:
    - "PLANNER" 키워드 → PLANNER 에이전트 추가
    - "NOTEWRITER" 또는 "note" → NOTEWRITER 추가
    - "ADVISOR" 또는 "guidance" → ADVISOR 추가
    
    병렬 실행 규칙:
    - PLANNER + NOTEWRITER: 동시 실행 가능
    - ADVISOR: 보통 순차 실행
    """
    try:
        # 기본 설정 (항상 PLANNER 포함)
        analysis = {
            "required_agents": ["PLANNER"],
            "priority": {"PLANNER": 1},
            "concurrent_groups": [["PLANNER"]],
            "reasoning": response
        }
        
        # ReACT 패턴 확인
        if "Thought:" in response and "Decision:" in response:
            # NOTEWRITER 필요 여부 확인
            if "NOTEWRITER" in response or "note" in response.lower():
                analysis["required_agents"].append("NOTEWRITER")
                analysis["priority"]["NOTEWRITER"] = 2
                # PLANNER와 병렬 실행
                analysis["concurrent_groups"] = [["PLANNER", "NOTEWRITER"]]
            
            # ADVISOR 필요 여부 확인
            if "ADVISOR" in response or "guidance" in response.lower():
                analysis["required_agents"].append("ADVISOR")
                analysis["priority"]["ADVISOR"] = 3
                # ADVISOR는 보통 순차 실행
        
        return analysis
    
    except Exception as e:
        print(f"⚠️  파싱 오류: {str(e)}")
        # 안전한 기본값 반환
        return {
            "required_agents": ["PLANNER"],
            "priority": {"PLANNER": 1},
            "concurrent_groups": [["PLANNER"]],
            "reasoning": "파싱 오류로 인한 폴백"
        }

print("✅ Coordinator Agent 정의 완료")

In [ ]:
class PlannerAgent(ReActAgent):
    """
    일정 계획을 수립하는 전문 에이전트
    
    ReActAgent를 상속받아 기본 tools(search_calendar 등)를 사용하고,
    전문 프롬프트로 LLM을 호출합니다.
    """
    
    def __init__(self, llm):
        """부모 클래스 초기화 (tools 상속)"""
        super().__init__(llm)
        self.llm = llm
    
    async def __call__(self, state: AcademicState) -> Dict:
        """
        PlannerAgent의 메인 실행 로직
        
        처리 흐름:
        1. 캘린더와 할 일 목록 수집 (상속받은 tools 사용)
        2. 학생 프로필과 학습 스타일 조회
        3. LLM에게 계획 수립 요청
        4. 결과 반환
        """
        try:
            # 1. 데이터 수집 (상속받은 tools 사용)
            upcoming_events = await self.search_calendar(state)
            tasks = await self.analyze_tasks(state)
            
            # 2. 학생 프로필 정보
            profile = state.get("profile", {})
            learning_style = profile.get("learning_preferences", {}).get("learning_style", {})
            
            # 3. 사용자 요청
            user_request = state["messages"][-1].content
            
            # 4. 프롬프트 구성
            planner_prompt = f"""당신은 학생의 학업 계획을 수립하는 전문가입니다.

학생 요청: {user_request}

학생 정보:
- 학습 스타일: {learning_style.get('primary_style', 'Unknown')}
- 선호 학습 시간: {learning_style.get('preferred_times', [])}

다가오는 일정 ({len(upcoming_events)}개):
{json.dumps(upcoming_events[:3], ensure_ascii=False, indent=2)}  # 최대 3개만 표시

할 일 목록 ({len(tasks)}개):
{json.dumps(tasks[:3], ensure_ascii=False, indent=2)}  # 최대 3개만 표시

다음 형식으로 계획을 작성하세요:

Thought: [상황 분석]
Action: [필요한 작업 식별]
Observation: [현재 일정 및 과제 상태]
Plan: [구체적인 학습 계획]

계획에 포함할 내용:
- 우선순위가 높은 과제 식별
- 학습 시간 배분
- 학생의 학습 스타일에 맞는 전략
- 휴식 시간 포함
"""
            
            # 5. LLM 호출
            response = await llm_client.chat.completions.create(
                model=LLMConfig.MODEL_NAME,
                messages=[
                    {"role": "system", "content": planner_prompt}
                ],
                max_tokens=LLMConfig.MAX_TOKENS,
                temperature=LLMConfig.TEMPERATURE
            )
            
            plan = response.choices[0].message.content
            
            # 6. 결과 반환
            return {
                "plan": plan,
                "events_analyzed": len(upcoming_events),
                "tasks_analyzed": len(tasks)
            }
        
        except Exception as e:
            print(f"⚠️  PlannerAgent 오류: {e}")
            return {
                "plan": "계획 수립 중 오류가 발생했습니다. 나중에 다시 시도해주세요.",
                "error": str(e)
            }

# 간단한 NoteWriterAgent와 AdvisorAgent 스텁 (실제로는 더 복잡)
class NoteWriterAgent(ReActAgent):
    """학습 자료 작성 에이전트 (PlannerAgent와 유사한 구조)"""
    async def __call__(self, state: AcademicState) -> Dict:
        return {"note": "학습 노트가 여기에 생성됩니다."}

class AdvisorAgent(ReActAgent):
    """학습 조언 제공 에이전트 (PlannerAgent와 유사한 구조)"""
    async def __call__(self, state: AcademicState) -> Dict:
        return {"advice": "학습 조언이 여기에 생성됩니다."}

print("✅ 전문 에이전트 클래스 정의 완료")

In [ ]:
# ===================================
# 최종 응답 생성 노드
# ===================================

async def response_generator(state: AcademicState) -> Dict:
    """
    에이전트들의 결과를 통합하여 최종 사용자 응답 생성
    """
    try:
        agent_outputs = state["results"].get("agent_outputs", {})
        
        # 각 에이전트의 결과를 포맷팅
        response_parts = []
        
        if "planner" in agent_outputs:
            plan = agent_outputs["planner"].get("plan", "")
            response_parts.append(f"## 📅 학습 계획\n\n{plan}")
        
        if "notewriter" in agent_outputs:
            note = agent_outputs["notewriter"].get("note", "")
            response_parts.append(f"\n\n## 📝 학습 노트\n\n{note}")
        
        if "advisor" in agent_outputs:
            advice = agent_outputs["advisor"].get("advice", "")
            response_parts.append(f"\n\n## 💡 학습 조언\n\n{advice}")
        
        final_response = "\n".join(response_parts) if response_parts else "응답을 생성할 수 없습니다."
        
        return {"final_response": final_response}
    
    except Exception as e:
        return {"final_response": f"응답 생성 중 오류: {str(e)}"}

# ===================================
# LangGraph 워크플로우 구성
# ===================================

def create_atlas_workflow() -> StateGraph:
    """
    ATLAS 시스템의 전체 워크플로우 그래프 생성
    
    Returns:
        컴파일된 StateGraph (실행 가능한 워크플로우)
    """
    # 1. StateGraph 초기화
    workflow = StateGraph(AcademicState)
    
    # 2. AgentExecutor 인스턴스 생성
    executor = AgentExecutor(llm_client)
    
    # 3. 노드 추가
    workflow.add_node("coordinator", coordinator_agent)
    workflow.add_node("executor", executor.execute)
    workflow.add_node("response_generator", response_generator)
    
    # 4. 엣지 추가 (실행 순서 정의)
    workflow.add_edge("coordinator", "executor")
    workflow.add_edge("executor", "response_generator")
    
    # 5. 시작점 설정
    workflow.set_entry_point("coordinator")
    
    # 6. 종료점 설정
    workflow.add_edge("response_generator", END)
    
    # 7. 컴파일 (실행 가능하게 변환)
    return workflow.compile()

# 워크플로우 생성
atlas_workflow = create_atlas_workflow()

print("✅ ATLAS 워크플로우 생성 완료")
print("📊 노드: coordinator → executor → response_generator → END")

# 🎓 요약 및 다음 단계

## 학습한 내용:

### 1. 기본 구조
- **AcademicState**: 모든 에이전트가 공유하는 상태
- **DataManager**: 데이터 중앙 관리
- **LLMConfig**: LLM 설정 관리

### 2. ReACT 패턴
- **Thought**: 사고 (무엇을 해야 할까?)
- **Action**: 행동 (도구 사용)
- **Observation**: 관찰 (결과 확인)
- **반복**: 목표 달성까지 순환

### 3. 멀티 에이전트 시스템
- **Coordinator**: 요청 분석, 에이전트 선택
- **AgentExecutor**: 선택된 에이전트 실행
- **전문 에이전트들**: Planner, NoteWriter, Advisor

### 4. LangGraph 워크플로우
- **StateGraph**: 상태 기반 그래프
- **노드**: 처리 단계 (coordinator, executor, response)
- **엣지**: 실행 순서 정의

---

## 다음 학습 과제:

### 1. 코드 확장
- [ ] NoteWriterAgent의 실제 구현 추가
- [ ] AdvisorAgent의 실제 구현 추가
- [ ] 더 많은 도구(tools) 추가

### 2. 개선 사항
- [ ] 에러 처리 강화
- [ ] 로깅 시스템 추가
- [ ] 성능 모니터링

### 3. 고급 기능
- [ ] 대화 히스토리 관리
- [ ] 컨텍스트 윈도우 관리
- [ ] 스트리밍 응답

### 4. 실전 응용
- [ ] 실제 캘린더 API 연동 (Google Calendar)
- [ ] 할 일 관리 앱 연동 (Todoist, Notion)
- [ ] 웹 인터페이스 구축 (Streamlit, Gradio)

---

## 참고 자료:

- [LangGraph 공식 문서](https://langchain-ai.github.io/langgraph/)
- [LangChain 가이드](https://python.langchain.com/)
- [ReACT 패턴 논문](https://arxiv.org/abs/2210.03629)
- [NVIDIA NeMo](https://build.nvidia.com/)

---

## 🎉 축하합니다!

LangGraph 멀티 에이전트 시스템의 핵심 구조를 학습했습니다!

질문이나 피드백이 있으시면 언제든지 문의하세요.

---

In [ ]:
# ===================================
# 2. 워크플로우 실행 함수
# ===================================

async def run_atlas_example():
    """
    ATLAS 시스템 실행 예제
    """
    print("🚀 ATLAS 시스템 시작...\n")
    
    # 1. State 초기화
    initial_state = {
        "messages": [
            HumanMessage(content="다음 주 중간고사 준비 계획을 세워주세요.")
        ],
        "profile": sample_profile,
        "calendar": sample_calendar,
        "tasks": sample_tasks,
        "results": {}
    }
    
    print("📝 사용자 요청:", initial_state["messages"][0].content)
    print("\n" + "="*60 + "\n")
    
    # 2. 워크플로우 실행
    try:
        result = await atlas_workflow.ainvoke(initial_state)
        
        # 3. 결과 출력
        print("\n" + "="*60)
        print("\n📊 최종 응답:\n")
        print(result.get("final_response", "응답 없음"))
        print("\n" + "="*60)
        
        return result
    
    except Exception as e:
        print(f"\n⚠️  실행 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

# 실행 (주석 제거하여 실행)
# await run_atlas_example()

print("\n💡 실행 방법:")
print("   await run_atlas_example()")
print("\n⚠️  주의: API 키가 설정되어 있어야 합니다!")

In [ ]:
# ===================================
# 1. 샘플 데이터 준비
# ===================================

# 학생 프로필 샘플
sample_profile = {
    "id": "student001",
    "name": "김철수",
    "academic_info": {
        "major": "컴퓨터공학",
        "year": 3,
        "current_courses": [
            {
                "name": "데이터구조",
                "code": "CS201",
                "credits": 3
            },
            {
                "name": "알고리즘",
                "code": "CS301",
                "credits": 3
            }
        ]
    },
    "learning_preferences": {
        "learning_style": {
            "primary_style": "visual",
            "preferred_times": ["morning", "afternoon"]
        },
        "study_patterns": {
            "session_length": 50,
            "break_length": 10
        }
    }
}

# 캘린더 샘플
sample_calendar = {
    "events": [
        {
            "summary": "데이터구조 중간고사",
            "start": {
                "dateTime": "2025-11-01T10:00:00Z"
            },
            "description": "중간고사"
        },
        {
            "summary": "알고리즘 과제 제출",
            "start": {
                "dateTime": "2025-10-28T23:59:00Z"
            },
            "description": "과제"
        }
    ]
}

# 할 일 목록 샘플
sample_tasks = {
    "tasks": [
        {
            "title": "알고리즘 과제 3",
            "due": "2025-10-28T23:59:00Z",
            "status": "needsAction",
            "course": "알고리즘"
        },
        {
            "title": "데이터구조 복습",
            "due": "2025-10-31T23:59:00Z",
            "status": "needsAction",
            "course": "데이터구조"
        }
    ]
}

print("✅ 샘플 데이터 준비 완료")

# 🚀 12. 실행 예제

**핵심 개념**: 전체 시스템을 실제로 실행해봅니다.

## 준비물:
1. 학생 프로필 JSON
2. 캘린더 데이터 JSON
3. 할 일 목록 JSON

## 실행 단계:
1. 샘플 데이터 준비
2. AcademicState 초기화
3. 워크플로우 실행
4. 결과 확인

---

# 🔄 11. LangGraph 워크플로우 (전체 시스템 연결)

**핵심 개념**: LangGraph로 Coordinator → AgentExecutor → FinalResponse 흐름을 정의합니다.

## 워크플로우 구조:

```
START
  ↓
coordinator_node  ← 요청 분석, 에이전트 선택
  ↓
executor_node     ← 선택된 에이전트들 실행
  ↓
response_node     ← 결과 통합 및 사용자 응답 생성
  ↓
END
```

## StateGraph의 역할:
- **노드**: 각 처리 단계 (coordinator, executor, response)
- **엣지**: 노드 간 연결 (실행 순서)
- **State**: 모든 노드가 공유하는 데이터 (AcademicState)

## 장점:
- 명확한 데이터 흐름
- 각 단계 독립적으로 테스트 가능
- 시각화 가능 (graph.get_graph().draw_mermaid())

---

In [ ]:
class AgentExecutor:
    """
    여러 에이전트를 병렬/순차로 실행하는 실행기
    
    Coordinator의 분석 결과를 바탕으로 에이전트들을 조율합니다.
    """
    
    def __init__(self, llm):
        """전문 에이전트들 초기화"""
        self.llm = llm
        self.agents = {
            "PLANNER": PlannerAgent(llm),
            "NOTEWRITER": NoteWriterAgent(llm),
            "ADVISOR": AdvisorAgent(llm)
        }
    
    async def execute(self, state: AcademicState) -> Dict:
        """
        Coordinator 분석을 바탕으로 에이전트 실행
        
        처리 흐름:
        1. Coordinator 분석 결과 읽기
        2. concurrent_groups별로 병렬 실행
        3. 결과 수집 및 반환
        """
        try:
            # 1. Coordinator 분석 결과 추출
            analysis = state["results"].get("coordinator_analysis", {})
            required_agents = analysis.get("required_agents", ["PLANNER"])
            concurrent_groups = analysis.get("concurrent_groups", [["PLANNER"]])
            
            print(f"📋 실행할 에이전트: {required_agents}")
            print(f"🔄 병렬 그룹: {concurrent_groups}")
            
            # 2. 결과 저장소
            results = {}
            
            # 3. 각 그룹별로 순차 실행 (그룹 내에서는 병렬)
            for group in concurrent_groups:
                # 그룹 내 에이전트들의 태스크 준비
                tasks = []
                for agent_name in group:
                    if agent_name in required_agents and agent_name in self.agents:
                        tasks.append(self.agents[agent_name](state))
                
                # 병렬 실행 (asyncio.gather)
                if tasks:
                    print(f"⚡ {group} 실행 중...")
                    group_results = await asyncio.gather(*tasks, return_exceptions=True)
                    
                    # 성공한 결과만 저장
                    for agent_name, result in zip(group, group_results):
                        if not isinstance(result, Exception):
                            results[agent_name.lower()] = result
                            print(f"  ✅ {agent_name} 완료")
                        else:
                            print(f"  ⚠️  {agent_name} 오류: {result}")
            
            # 4. 폴백: 결과가 없으면 PLANNER 실행
            if not results and "PLANNER" in self.agents:
                print("⚠️  폴백: PLANNER 실행")
                planner_result = await self.agents["PLANNER"](state)
                results["planner"] = planner_result
            
            # 5. 결과 반환
            return {
                "results": {
                    "agent_outputs": results
                }
            }
        
        except Exception as e:
            print(f"⚠️  AgentExecutor 오류: {e}")
            return {
                "results": {
                    "agent_outputs": {
                        "planner": {
                            "plan": "시스템 오류가 발생했습니다. 다시 시도해주세요."
                        }
                    }
                }
            }

print("✅ AgentExecutor 클래스 정의 완료")

# ⚙️ 10. AgentExecutor (에이전트 실행기)

**핵심 개념**: AgentExecutor는 Coordinator가 선택한 에이전트들을 실제로 실행합니다.

## 실행 흐름:

```
Coordinator → "PLANNER와 NOTEWRITER 병렬 실행"
         ↓
AgentExecutor:
  1. Coordinator 분석 결과 읽기
  2. concurrent_groups 확인
  3. asyncio.gather()로 병렬 실행
  4. 결과 수집 및 반환
```

## 주요 기능:
- **병렬 실행**: 독립적인 에이전트는 동시에 실행 (성능 향상)
- **에러 처리**: 한 에이전트 실패해도 다른 에이전트는 계속 실행
- **폴백**: 모든 에이전트 실패 시 PLANNER로 폴백

---

# 📅 9. PlannerAgent (계획 에이전트 예시)

**핵심 개념**: PlannerAgent는 ReActAgent를 상속받아 일정 관리 기능을 구현합니다.

## 전문 에이전트의 구조:

```python
class PlannerAgent(ReActAgent):  # ReActAgent 상속
    def __init__(self, llm):
        super().__init__(llm)  # 부모 초기화 (tools 상속)
        self.prompt = "..."     # 전문 프롬프트 정의
    
    async def __call__(self, state):  # 실행 로직
        # 1. 데이터 수집 (상속받은 tools 사용)
        # 2. LLM 호출 (self.llm)
        # 3. 결과 반환
```

## PlannerAgent의 역할:
- 캘린더 이벤트 분석
- 할 일 목록 우선순위 설정
- 학습 시간 배분 계획
- 학생의 학습 스타일 고려

## 다른 전문 에이전트:
- **NoteWriterAgent**: 학습 자료 작성 (PlannerAgent와 유사한 구조)
- **AdvisorAgent**: 학습 조언 제공 (PlannerAgent와 유사한 구조)

---

# 🎯 8. Coordinator Agent (조율 에이전트)

**핵심 개념**: Coordinator는 시스템의 "두뇌"입니다. 사용자 요청을 분석하고 적절한 에이전트에게 작업을 위임합니다.

## Coordinator의 역할:

```
사용자: "다음 주 시험 준비 계획 세워줘"
         ↓
Coordinator (분석):
  - Thought: "시험 준비 = 계획 + 학습 조언 필요"
  - Action: "PLANNER와 ADVISOR 호출"
  - Decision: "두 에이전트 병렬 실행 가능"
         ↓
  ┌────────┴────────┐
  ↓                 ↓
PLANNER          ADVISOR
(일정 작성)      (학습 조언)
```

## 관리하는 에이전트:
- **PLANNER**: 일정 관리 및 시간 배분
- **NOTEWRITER**: 학습 자료 작성 및 요약
- **ADVISOR**: 학습 전략 및 조언

## 핵심 기능:
1. **analyze_context()**: 학생 상황 분석 (프로필, 일정, 과제 등)
2. **coordinator_agent()**: ReACT 패턴으로 에이전트 선택
3. **parse_coordinator_response()**: LLM 응답 파싱 및 구조화

## 병렬 실행 전략:
- 독립적인 에이전트는 동시 실행 (성능 향상)
- 의존성 있는 작업은 순차 실행
- 에러 발생 시 PLANNER로 폴백

---

# 🧠 7. ReActAgent (기본 에이전트)

**핵심 개념**: ReACT 패턴을 구현하는 기본 에이전트 클래스입니다.

## ReACT 패턴이란?
**Re**asoning (추론) + **Act**ing (행동)을 결합한 패턴

```
1. Thought (사고): "무엇을 해야 할까?"
   ↓
2. Action (행동): "캘린더를 검색하자"
   ↓
3. Observation (관찰): "다음 주에 3개 시험 발견"
   ↓
(반복)
```

## ReActAgent의 핵심 기능:
- **Tools**: 에이전트가 사용할 수 있는 도구 모음
  - `search_calendar()`: 캘린더 검색
  - `analyze_tasks()`: 과제 분석
  - `check_learning_style()`: 학습 스타일 확인
  - `check_performance()`: 성적 확인

## 왜 상속하나?
- PlannerAgent, NoteWriterAgent, AdvisorAgent는 모두 ReActAgent를 상속
- 공통 기능(tools)은 재사용, 전문 기능만 각자 구현

---

In [ ]:
class AgentAction(BaseModel):
    """
    에이전트의 행동 결정을 표현하는 모델
    
    ReACT 패턴의 'Act' 부분에 해당합니다.
    """
    # 필수: 수행할 작업 (예: "search_calendar", "analyze_tasks")
    action: str
    
    # 필수: 왜 이 작업을 선택했는지 (사고 과정)
    thought: str
    
    # 선택: 사용할 도구 이름
    tool: Optional[str] = None
    
    # 선택: 작업에 필요한 입력 데이터
    action_input: Optional[Dict] = None

class AgentOutput(BaseModel):
    """
    에이전트 작업의 결과를 표현하는 모델
    
    ReACT 패턴의 'Observation' 부분에 해당합니다.
    """
    # 관찰 결과 (텍스트 설명)
    observation: str
    
    # 구조화된 출력 데이터
    output: Dict

print("✅ Agent Models 정의 완료")

# 🎯 6. Agent Models (에이전트 데이터 모델)

**핵심 개념**: Pydantic을 사용해 에이전트의 입력/출력 데이터 구조를 정의합니다.

## AgentAction:
- 에이전트가 수행할 작업 정의
- `thought`: 왜 이 작업을 하는지 (사고 과정)
- `action`: 무엇을 할 것인지 (행동)
- `tool`: 어떤 도구를 사용할지 (선택사항)

## AgentOutput:
- 작업 수행 결과
- `observation`: 관찰한 내용 (텍스트 설명)
- `output`: 구조화된 결과 데이터 (딕셔너리)

## 왜 Pydantic을 사용하나?
- 타입 안전성: 잘못된 데이터 입력 방지
- 자동 검증: 데이터가 유효한지 자동 확인
- 명확한 구조: 코드 가독성 향상

---

# 📦 5. DataManager (데이터 관리자)

**핵심 개념**: DataManager는 학생 프로필, 캘린더, 할 일 목록 등의 데이터를 중앙에서 관리합니다.

## 주요 기능:
- `load_data()`: JSON 데이터 로드 및 파싱
- `get_student_profile()`: 학생 ID로 프로필 검색
- `parse_datetime()`: 타임존을 고려한 날짜/시간 파싱
- `get_upcoming_events()`: 향후 N일간의 일정 조회
- `get_active_tasks()`: 미완료 과제 목록 조회

## 왜 필요한가?
- 에이전트들이 일관된 방식으로 데이터에 접근
- 타임존 처리 등 복잡한 로직을 한 곳에서 관리
- 데이터 소스가 변경되어도 에이전트 코드는 수정 불필요

---

# 🤖 4. LLM 설정 (NVIDIA NeMo)

NVIDIA의 Nemotron-4 340B 모델을 사용합니다. OpenAI 호환 API를 제공하므로 익숙한 인터페이스로 사용 가능합니다.

## LLMConfig 클래스:
- API 엔드포인트 설정
- 모델 이름 지정
- 최대 토큰 수, Temperature 등 파라미터 관리

---

In [ ]:
# TypeVar: 제네릭 타입 정의용
T = TypeVar('T')

class AcademicState(TypedDict):
    """
    학업 관리 시스템의 전체 상태
    
    모든 에이전트가 이 상태를 공유하며, 각자 필요한 정보를 읽고 업데이트합니다.
    """
    # 대화 히스토리 (HumanMessage, SystemMessage 등)
    messages: Annotated[List[BaseMessage], operator.add]
    
    # 학생 프로필 (JSON 형태)
    profile: Dict[str, Any]
    
    # 캘린더 이벤트 (JSON 형태)
    calendar: Dict[str, Any]
    
    # 할 일 목록 (JSON 형태)
    tasks: Dict[str, Any]
    
    # 현재 작동 중인 에이전트 이름
    current_agent: str
    
    # 각 에이전트의 출력 결과
    agent_outputs: Dict[str, str]
    
    # 최종 응답
    final_response: str

print("✅ AcademicState 정의 완료")

# 📊 3. State 정의 (상태 관리)

**핵심 개념**: LangGraph에서 State는 모든 에이전트가 공유하는 데이터 저장소입니다.

## AcademicState 구조:
- `messages`: 대화 히스토리
- `profile`: 학생 프로필 (학습 스타일, 전공 등)
- `calendar`: 일정 데이터
- `tasks`: 할 일 목록
- `current_agent`: 현재 작동 중인 에이전트
- `agent_outputs`: 각 에이전트의 출력 결과

각 에이전트는 이 State를 읽고 업데이트합니다.

---

In [ ]:
# API 키 설정
NEMOTRON_4_340B_INSTRUCT_KEY = None  # 전역 변수 초기화

def configure_api_keys():
    """
    API 키를 설정하는 함수
    
    Colab 환경에서는 userdata.get()을 사용하고,
    로컬 환경에서는 직접 입력하거나 환경 변수를 사용합니다.
    """
    global NEMOTRON_4_340B_INSTRUCT_KEY
    
    # Colab 환경
    try:
        from google.colab import userdata
        NEMOTRON_4_340B_INSTRUCT_KEY = userdata.get('NEMOTRON_4_340B_INSTRUCT_KEY')
        print("✅ Colab에서 API 키 로드 완료")
    except:
        # 로컬 환경: 직접 입력
        NEMOTRON_4_340B_INSTRUCT_KEY = "your-api-key-here"
        print("⚠️  로컬 환경: API 키를 직접 입력하세요")
    
    return NEMOTRON_4_340B_INSTRUCT_KEY

# API 키 설정 실행
api_key = configure_api_keys()

# 🔑 2. API 키 설정

NVIDIA API 키를 설정합니다. [NVIDIA NGC](https://build.nvidia.com)에서 무료 API 키를 발급받을 수 있습니다.

**중요**: API 키는 절대 공개 저장소에 업로드하지 마세요!

---

In [ ]:
# ============================================
# 필수 라이브러리 임포트
# ============================================

# 기본 유틸리티
import operator
from functools import reduce
from typing import Annotated, List, Dict, TypedDict, Literal, Optional, Callable, Set, Tuple, Any, Union, TypeVar
from datetime import datetime, timezone, timedelta
import asyncio
import json
import re
import os

# 데이터 검증 (Pydantic)
from pydantic import BaseModel, Field

# OpenAI 호환 API (NVIDIA NeMo 사용)
from openai import OpenAI, AsyncOpenAI

# LangChain 핵심
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# LangGraph (워크플로우 그래프)
from langgraph.graph import StateGraph, Graph, END, START

# 출력 포맷팅 (Rich)
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.text import Text
from rich import box
from rich.style import Style

print("✅ 모든 라이브러리 임포트 완료")

In [ ]:
# 패키지 설치 (Colab 환경에서 실행)
# 주석 해제하여 실행하세요

# !pip install langgraph langchain langchain-openai openai python-dotenv
# !pip install graphviz pygraphviz rich

print("✅ 패키지 설치 준비 완료")

# 📦 1. 패키지 설치 및 라이브러리 임포트

## 필수 라이브러리 설치

먼저 필요한 라이브러리들을 설치합니다. 각 라이브러리의 역할은 다음과 같습니다:

### 핵심 라이브러리:
- **`langgraph`**: AI 워크플로우를 그래프로 구성하는 프레임워크
- **`langchain`**: LLM 애플리케이션 개발 프레임워크  
- **`openai`**: OpenAI API 호환 인터페이스 (NVIDIA NeMo 사용)
- **`pydantic`**: 데이터 검증 및 타입 안전성 보장

### 시각화 라이브러리:
- **`graphviz`**: 그래프 시각화
- **`rich`**: 터미널 출력 포맷팅

---